In [207]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
class Model:  
    """
    This class represents an AI model.
    """
    
    def __init__(self):
        self.model = None
    def fillNanValues(self, image):
        c, h, w = image.shape
        nan_mask = np.isnan(image)
        # Find indices of NaN values
        nan_indices = np.where(nan_mask)
        # Iterate through each NaN pixel and replace it with the mean of its neighbors
        for idx in range(len(nan_indices[0])):
            y, x = nan_indices[1][idx], nan_indices[2][idx]
            # Extract the 8 neighboring pixels for each channel without wrapping around
            for channel in range(c):
                valid_neighbors = [
                    image[channel, y2, x2]
                    for y2 in range(max(0, y - 1), min(h, y + 2))
                    for x2 in range(max(0, x - 1), min(w, x + 2))
                    if (y2, x2) != (y, x)
                ]
                # Calculate the mean of the valid neighbors and replace the NaN value for each channel
                image[channel, y, x] = np.nanmean(valid_neighbors)
        return image
    def oversample(self, images, labels):
        minority_labels = [1, 2]
        # Find the number of instances of label 0
        num_instances_label_0 = np.sum(labels == 0)
        for minority_label in minority_labels:
            minority_indices = np.where(labels == minority_label)[0]
            num_instances_to_add = num_instances_label_0 - len(minority_indices)
            sampled_minority_indices = np.random.choice(minority_indices, size=num_instances_to_add, replace=True)
            for idx in sampled_minority_indices:
                # Assuming data is in the shape (C, H, W)
                image_data = torch.tensor(images[idx])
                if np.random.rand() < 0.3:
                    image_data = torch.flip(image_data, dims=[2])
                # Apply random vertical flip
                if np.random.rand() < 0.3:
                    image_data = torch.flip(image_data, dims=[1])
                # Append the augmented data to the original dataset
                images = np.concatenate([images, [image_data.numpy()]], axis=0)
                labels = np.concatenate([labels, [minority_label]])
        
        return images, labels
    
    def preprocess(self, X, y= None):
        if y is not None:
            # Remove NaN labels
            nan_mask = np.isnan(y)
            labels = y[~nan_mask]
            images = X[~nan_mask]
            images[images < 0] = 0  # Set negative pixel values to 0
            images[images > 255] = 255  # Set pixel values > 1 to 1
            # Impute missing values
            images = np.array([self.fillNanValues(img) for img in images])
            # Oversample to handle imbalance through augmentation
            images, labels = self.oversample(images, labels)
            return (images, labels)
        
        else:
            print("Preprocessing test data")
            X[X < 0] = 0  # Set negative pixel values to 0
            X[X > 255] = 255
            images = np.array([self.fillNanValues(img) for img in X])
            return (images)
    
    def fit(self, X, y):
        images, labels = self.preprocess(X, y)
        nan_count_dim0 = np.sum(np.isnan(images), axis=0)
        images_train = torch.tensor(images, dtype=torch.float32)
        y_train_tensor = torch.tensor(labels, dtype=torch.long)
        train_dataset = TensorDataset(images_train, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        class CNN(nn.Module):
            def __init__(self, num_classes, dropout_rate=0.2):
                super(CNN, self).__init__()
                self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
                self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
                self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
                
                self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
                self.act = nn.LeakyReLU(negative_slope=0.02)
                self.dropout = nn.Dropout2d(p=dropout_rate)
                self.flatten = nn.Flatten()
                self.fc1 = nn.Linear(128 * 2 * 2, num_classes)  # New linear layer
                # self.relu = nn.ReLU()  # ReLU activation
                # self.fc2 = nn.Linear(256, num_classes)
            def forward(self, x):
                x = self.dropout(self.pool(self.act(self.conv1(x))))
                x = self.dropout(self.pool(self.act(self.conv2(x))))
                x = self.dropout(self.pool(self.act(self.conv3(x))))
                x = self.flatten(x)
                # Apply the new linear layer with ReLU activation
                x = self.fc1(x)
                return x
        # Instantiate the model
        num_classes = 3  # Change this based on your dataset
        self.model = CNN(num_classes)
        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        # Training the model
        num_epochs = 30
        for _ in range(num_epochs):
            self.model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        return self
    
    def predict(self, X):
        # Preprocess the test data
        X = self.preprocess(X)
        
        X_tensor = torch.tensor(X, dtype=torch.float32)
        # Make predictions using the trained neural network
        with torch.no_grad():
            outputs = self.model(X_tensor)
        _, predicted_labels = torch.max(outputs, 1)
        # Convert PyTorch tensor to numpy array
        return predicted_labels.numpy()


#### Local Evaluation

You may test your solution locally by running the following code. Do note that the results may not reflect your performance in Coursemology. You should not be submitting the code below in Coursemology. The code here is meant only for you to do local testing.

In [198]:
# Import packages
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

In [199]:
# Load data
with open('data.npy', 'rb') as f:
    data = np.load(f, allow_pickle=True).item()
    X = data['image']
    y = data['label']